# <b>Beam Sarch Exploration</b>

### <b>Greedy Decoder</b>. Picks the word with <b>highest probability</b> from a vocabulary

In [3]:
import numpy as np
from numpy import array, argmax

# greedy decoder
def greedy_decoder(data):
    # index for largest probability each row
    return [argmax(s) for s in data]


#### Example: assume "data" is the result of a prediction task that predicts the next 10 words given the previous word over a vocab of 5 words.

I start from a prediciton that tolds me that word1 is probable at 0.1, word2 at 0.2, ..., word3 at 0.5

The greedy decoder returns the vocabulary entries (indexes pointing to vocabulary) with highest probability for each word

In [4]:
vocab = {0:'pippo', 1:'sandrino', 2:'pieroni', 3:'marzullo', 4:'catorcio'}

In [5]:
#np.random.uniform(0, 1, 5)

In [6]:
def random_data(sequence_size, vocab_size):
    result = []
    for i in range(0, sequence_size):
        result.append(np.random.uniform(0, 1, vocab_size))
    return array(result)

In [7]:
data = random_data(10,5)

print(data)

[[ 0.52457992  0.28552338  0.75380654  0.46466122  0.68603885]
 [ 0.73735901  0.61122364  0.86520015  0.42268385  0.68092913]
 [ 0.59109473  0.64146109  0.59287713  0.4934406   0.4659128 ]
 [ 0.05324533  0.21668017  0.36318025  0.71377578  0.13925981]
 [ 0.18826333  0.29045033  0.31112292  0.75633354  0.69905173]
 [ 0.45696941  0.916494    0.03803797  0.91780142  0.92194863]
 [ 0.36227624  0.91316214  0.14919399  0.93794889  0.88271121]
 [ 0.35747485  0.5084773   0.11020641  0.54787643  0.06448803]
 [ 0.60437343  0.94158365  0.06899962  0.85873245  0.01354531]
 [ 0.62594457  0.93824783  0.99628312  0.51376727  0.23904859]]


In [8]:
# decode sequence
result = greedy_decoder(data)

print(result)

[2, 2, 1, 3, 3, 4, 3, 3, 1, 2]


In [9]:
# decode highest probability word indexes to vocab words
def decode_from_vocab(vocab, idx_array):
    decoded = []
    for idx in idx_array:
        decoded.append(vocab[idx])
    return decoded

In [10]:
decode_from_vocab(vocab, result)

['pieroni',
 'pieroni',
 'sandrino',
 'marzullo',
 'marzullo',
 'catorcio',
 'marzullo',
 'marzullo',
 'sandrino',
 'pieroni']

### <b>Beam Search Decoder</b>. Expands all possible next steps and keeps the <b>k most likely</b> (where k controls the number of beams or parallel searches through the sequence of probabilities)


*The local beam search algorithm keeps track of k states rather than just one. It begins with k randomly generated states. At each step, all the successors of all k states are generated. If any one is a goal, the algorithm halts. Otherwise, it selects the k best successors from the complete list and repeats.*

Common beam width values are 1 for a greedy search and values of 5 or 10 for common benchmark problems in machine translation

In [20]:
from math import log

# beam search
def beam_search_decoder(data, k):
    print("k = ", k)
    sequences = [[list(), 1.0]]
    # walk over each step in sequence
    for row in data:
        print("row", row)
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            print("seq:", seq, "seq score:", score, "")
            for j in range(len(row)):
                candidate = [seq + [j], score * -log(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
        print("k candidates:", sequences)
    return sequences

#### Example: assume "data" is the result of a prediction task that predicts the next 10 words given the previous word over a vocab of 5 words. Beam search performed on predicted data with k = 3


In [24]:
data = random_data(10,5)
print(data)

[[ 0.08398073  0.10248591  0.02625078  0.66080139  0.08036359]
 [ 0.95381613  0.20863821  0.9569603   0.79854756  0.12084853]
 [ 0.01291866  0.35305058  0.64047748  0.78512149  0.40451399]
 [ 0.10412229  0.86880292  0.13211371  0.32664095  0.5470383 ]
 [ 0.53654872  0.00516611  0.86217385  0.30588989  0.39414109]
 [ 0.46442831  0.96799058  0.98349889  0.0509769   0.11643417]
 [ 0.90624877  0.7578022   0.85447125  0.41805789  0.6897766 ]
 [ 0.66172551  0.5723441   0.59171137  0.16374587  0.19091871]
 [ 0.89254568  0.35856224  0.24766143  0.07932327  0.71868108]
 [ 0.80857581  0.86766567  0.1879227   0.11962485  0.30550122]]


In [25]:
# decode sequence
beam_result = beam_search_decoder(data, 3)
# print result
for seq in beam_result:
    print(seq)
    print(decode_from_vocab(vocab, seq[0]))

k =  3
row [ 0.08398073  0.10248591  0.02625078  0.66080139  0.08036359]
seq: [] seq score: 1.0 
k candidates: [[[3], 0.41430195145959264], [[1], 2.2780299905206287], [[0], 2.4771678613483017]]
row [ 0.95381613  0.20863821  0.9569603   0.79854756  0.12084853]
seq: [3] seq score: 0.41430195145959264 
seq: [1] seq score: 2.2780299905206287 
seq: [0] seq score: 2.4771678613483017 
k candidates: [[[3, 2], 0.018226541464463], [[3, 0], 0.01959000398735612], [[3, 3], 0.0932016799241813]]
row [ 0.01291866  0.35305058  0.64047748  0.78512149  0.40451399]
seq: [3, 2] seq score: 0.018226541464463 
seq: [3, 0] seq score: 0.01959000398735612 
seq: [3, 3] seq score: 0.0932016799241813 
k candidates: [[[3, 2, 3], 0.004409306753330751], [[3, 0, 3], 0.004739151256295161], [[3, 2, 2], 0.008120677240923387]]
row [ 0.10412229  0.86880292  0.13211371  0.32664095  0.5470383 ]
seq: [3, 2, 3] seq score: 0.004409306753330751 
seq: [3, 0, 3] seq score: 0.004739151256295161 
seq: [3, 2, 2] seq score: 0.008120677